<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/25411445/" target="__blank">Mignot, et al. (2015) WWOX-related encephalopathies: delineation of the phenotypical spectrum and emerging genotype-phenotype correlation</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was copied from table 1 in the original publication.</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.6.3


<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
pmid = "PMID:25411445"
title = "WWOX-related encephalopathies: delineation of the phenotypical spectrum and emerging genotype-phenotype correlation"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=pmid, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [6]:
df = pd.read_excel('./input/PMID_25411445.xlsx')

Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that. Furthermore, remove individuals without an age specified or a variant in this gene.

In [7]:
df = df.set_index('Patient').T.reset_index()
df['Age'] = 0
df['patient_id'] = df['index'] - 1
df.head()

Patient,index,Sex,Age,Mutation,DNA level (NM_016373.2),Second mutation,Protein level (NM_057457),Microcephaly,Profound global developmental delay,Axial hypotonia,...,Hypokinesia,Rigidity,Thin corpus callosum,Delayed myelination,Cerebral atrophy,Seizure,EEG abnormality,Reduced eye contact,Sudden death,patient_id
0,1,F,0,CH for two deletions of several exons,c.366_516del,c.517-?_1056+ ?del,p.[0];[His173_Met352del],Yes,Yes,Yes,...,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,0
1,2,F,0,CH: deletion of exon 6/ nonsense exon 8,c.1005G>A,c.517-?_605+?del,p.[His173Alafs*67];[Trp335*],Yes,Yes,Yes,...,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,1
2,3,M,0,CH: frameshift exon,c.45_48delGGAC,c.140C>G,p.[Asp16Serfs*63];[Pro47Arg],No,Yes,Yes,...,No,No,No,No,No,Yes,Yes,No,No,2
3,4,F,0,1/missense exon 2,c.45_48delGGAC,c.140C>G,NaN,No,Yes,Yes,...,No,No,No,No,No,Yes,Yes,No,No,3
4,5,F,0,CH: complete deletion/ nonsense exon 8,c.889A>T,c.-366-? *871+?del,p.[0];[Lys297*],No,Yes,Yes,...,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes,4


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [8]:
generator = SimpleColumnMapperGenerator(df=df, observed='+', excluded='-', hpo_cr=hpo_cr)
column_mappers_d = generator.try_mapping_columns()


In [9]:
print(generator.get_mapped_columns())

['Microcephaly', 'Profound global developmental delay', 'Axial hypotonia', 'Cerebellar ataxia', 'Spasticity', 'Hypokinesia', 'Rigidity', 'Thin corpus callosum', 'Delayed myelination', 'Cerebral atrophy', 'Seizure', 'EEG abnormality', 'Reduced eye contact', 'Sudden death']


In [10]:
print(generator.get_unmapped_columns())

['index', 'Sex', 'Age', 'Mutation', 'DNA level (NM_016373.2)', 'Second mutation', 'Protein level (NM_057457)', 'patient_id']


<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [11]:
genome = 'hg38'
default_genotype = 'compound heterozygous'
WWOX_transcript='NM_016373.2'
vvalidator = VariantValidator(genome_build=genome, transcript=WWOX_transcript)

TypeError: VariantColumnMapper.__init__() got an unexpected keyword argument 'assembly'

In [ ]:
varMapper = VariantColumnMapper(assembly=genome,column_name='DNA level (NM_016373.2)', 
                                transcript=transcript)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [38]:
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(df['Age'])

,original column contents,age
0,0,P0Y


In [39]:
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='Sex')
sexMapper.preview_column(df['Sex'])

,original column contents,sex
0,F,FEMALE
1,F,FEMALE
2,M,MALE
3,F,FEMALE
4,F,FEMALE


In [40]:

encoder = CohortEncoder(df=df, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="index", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='616211', label='DEE28')

In [41]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.366_516del/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.1005G>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.45_48delGGAC/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.45_48delGGAC/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.889A>T/NM_016373.2?content-type=application%2Fjson


In [42]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "0",
  "subject": {
    "id": "0",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P0Y"
      }
    },
    "sex": "FEMALE"
  },
  "interpretations": [
    {
      "id": "0",
      "progressStatus": "SOLVED",
      "diagnosis": {
        "disease": {
          "id": "616211",
          "label": "DEE28"
        },
        "genomicInterpretations": [
          {
            "subjectOrBiosampleId": "0",
            "interpretationStatus": "CAUSATIVE",
            "variantInterpretation": {
              "variationDescriptor": {
                "geneContext": {
                  "valueId": "HGNC:12799",
                  "symbol": "WWOX"
                },
                "expressions": [
                  {
                    "syntax": "hgvs.c",
                    "value": "NM_016373.2:c.366_516del"
                  },
                  {
                    "syntax": "hgvs.g",
                    "value": "NC_000016.9:g.78149008_78198186del"
          

In [43]:
output_directory = "../../phenopackets/WWOX/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.366_516del/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.1005G>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.45_48delGGAC/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.45_48delGGAC/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.889A>T/NM_016373.2?content-type=application%2Fjson
Wrote 5 phenopackets to ../../phenopackets/WWOX/
